### 🛠️ Configuração de device

In [1]:
import torch

n_devices = torch.cuda.device_count()

device_ = "cpu" if not n_devices else "cuda"

### 🗃️ Recuperando o log do notebook 01_EA_DF

In [2]:
import json
import os

In [3]:
path_video = "/home/elton/Projetos/Translater_Video/teste"
name_video = "1 - Preparations.mp4"

name_audio_out = "audio_completo.wav"
path_out = "/home/elton/Projetos/Translater_Video/teste/resultado_teste"
path_out_audios = "/home/elton/Projetos/Translater_Video/teste/resultado_teste/audios/gerados"
path_log = os.path.join(path_out, 'log', 'log.json')

In [4]:
log = ''
with open(path_log, 'r', encoding='utf-8') as file:
    log = json.load(file)

### 🎺 Remover acústica

In [5]:
from spleeter.separator import Separator
import soundfile as sf

# Inicializa separador em 2 stems (voz e instrumental)
separator = Separator('spleeter:2stems')

2025-08-27 11:45:19.702846: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-27 11:45:19.936185: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-27 11:45:22.771317: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


INFO:spleeter:File output/audio_completo/vocals.wav written succesfully
INFO:spleeter:File output/audio_completo/accompaniment.wav written succesfully
INFO:spleeter:File /home/elton/Projetos/Translater_Video/teste/resultado_teste/audio_completo/vocals.wav written succesfully
INFO:spleeter:File /home/elton/Projetos/Translater_Video/teste/resultado_teste/audio_completo/accompaniment.wav written succesfully


In [6]:
a = os.path.join(path_out , name_audio_out)

In [8]:
# Separa em memória
prediction = separator.separate_to_file(a, path_out)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Apply unet for vocals_spectrogram
INFO:tensorflow:Apply unet for accompaniment_spectrogram
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from pretrained_models/2stems/model


2025-08-27 11:49:51.074035: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-08-27 11:49:51.074142: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:04:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-08-27 11:49:51.074158: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


2025-08-27 11:49:52.883114: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [8623373,2]
	 [[{{node Placeholder/_1}}]]


### Concatenação 01
`Concatenação dos áudios que vieram gerados de uma única frase`

In [139]:
from pydub import AudioSegment

In [140]:
audios_posicionados = []

for index, i in enumerate(log):
    q_audios = len(i['path_audios_gerados'])
    
    if q_audios > 1:
        name = []
        concatenacao_audios = AudioSegment.empty()
        
        #Create Audio_Segments dinâmico
        for _, k in enumerate(i['path_audios_gerados']):
            
            name.append(list(k.keys()))
            path_audio_atual = list(k.items())[0]
            #list(k.keys())[0] = AudioSegment.from_file(path_audio_atual[1])
            
            concatenacao_audios += AudioSegment.from_file(path_audio_atual[1])

            os.remove(path_audio_atual[1])
            
        #print(name[0][0])
        path_f = os.path.join(path_out_audios, 'audio'+ name[0][0][6:] + '.wav')
        concatenacao_audios.export(path_f, format='wav')
        
        
        log[index]['path_audios_gerados'] = [{name[0][0]: path_f}]
        # print(name[0])


### Duração
`Validando a duração dos áudios gerados`

In [143]:
import wave

In [144]:
def leitura_wave (path):
    with wave.open(path, 'rb') as wave_file:
        frames = wave_file.getnframes()        # quantidade de frames (amostras)
        rate = wave_file.getframerate()        # taxa de amostragem (frames por segundo)
        duration = frames / float(rate)       # duração em segundos
    return duration

In [145]:
for log_idx, i in enumerate(log):
    duracao_audios = 0
    
    for path in i['path_audios_gerados']:
        for key, item in path.items():
            duracao_audios += leitura_wave(item)

    log[log_idx]['duracao_audio_gerado'] = round(float(duracao_audios), 2)
    log[log_idx]['duracao_audio_original'] = round(float(i['end'] - i['start']), 2)



### Concatenação 2
`Gerando o áudio para ser adicionar ao vídeo`

In [178]:
duracao_total_audio = ((log[-1]['end'] - log[-1]['start']) - log[-1]['duracao_audio_gerado']) + log[-1]['end']

In [179]:
audio = AudioSegment.empty()
base = AudioSegment.silent(duration= int(duracao_total_audio * 1000))

In [184]:
result = ''

for index, lista in enumerate(log):
    time_start = int(lista['start'] * 1000)
    time_end = int(lista['end'] * 1000)
    
    for _, item in enumerate(lista['path_audios_gerados']):
        
        if index == 0:
            audio = audio.from_file(list(item.items())[0][1])
            result = base.overlay(audio, position=time_start)
        else:
            time_start_anterior = int(log[index-1]['start'] * 1000)
            time_end_anterior = int(log[index-1]['end'] * 1000)
            
            if time_end_anterior > time_start:
                calc = time_end_anterior - time_start
                time_start += time_start
                
                audio = audio.from_file(list(item.items())[0][1])
                result = result.overlay(audio, position=time_start)
            
            else:
                audio = audio.from_file(list(item.items())[0][1])
                result = result.overlay(audio, position=time_start)
    

    path_f = os.path.join(path_out_audios, 'resultado.wav')
    result.export(path_f, format='wav')
    
    log[index]['audio_concatenado'] = path_f
        

In [185]:
with open(path_log, 'w', encoding='utf-8') as file:
    file.write(json.dumps(log, ensure_ascii=False, indent=2))